### Import necessary libraries

In [117]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
#pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup
#pip install lxml

In [118]:
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [119]:
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [120]:
df.shape

(86952, 6)

In [121]:
df.dtypes

date       object
county     object
state      object
fips      float64
cases       int64
deaths      int64
dtype: object

In [122]:
df.isnull().sum()

date         0
county       0
state        0
fips      1070
cases        0
deaths       0
dtype: int64

In [123]:
df = df.drop(['fips'],axis=1)
df.head()

,date,county,state,cases,deaths
0,2020-01-21,Snohomish,Washington,1,0
1,2020-01-22,Snohomish,Washington,1,0
2,2020-01-23,Snohomish,Washington,1,0
3,2020-01-24,Cook,Illinois,1,0
4,2020-01-24,Snohomish,Washington,1,0


In [127]:
df_pop = pd.read_excel('https://www2.census.gov/programs-surveys/popest/tables/2010-2019/counties/totals/co-est2019-annres.xlsx',skiprows=5,header=None,usecols=[0,12])
df_pop.head()

,0,12
0,".Autauga County, Alabama",55869.0
1,".Baldwin County, Alabama",223234.0
2,".Barbour County, Alabama",24686.0
3,".Bibb County, Alabama",22394.0
4,".Blount County, Alabama",57826.0


In [128]:
df_pop = df_pop.rename(columns={0:'County, State',12:'Population Est. 2019'})
df_pop['County, State'] = df_pop['County, State'].str.replace(".","")
df_pop['County'], df_pop['State'] = df_pop['County, State'].str.split(', ',1).str
df_pop.drop(['County, State'],axis=1,inplace=True)
df_pop.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  This is separate from the ipykernel package so we can avoid doing imports until


,Population Est. 2019,County,State
0,55869.0,Autauga County,Alabama
1,223234.0,Baldwin County,Alabama
2,24686.0,Barbour County,Alabama
3,22394.0,Bibb County,Alabama
4,57826.0,Blount County,Alabama


In [129]:
df_pop.shape

(3148, 3)

In [130]:
df_pop.dtypes

Population Est. 2019    float64
County                   object
State                    object
dtype: object

In [131]:
df_pop.isnull().sum()

Population Est. 2019    6
County                  0
State                   2
dtype: int64

In [136]:
df_popnull = df_pop[pd.isnull(df_pop).any(axis=1)]
df_popnull

,Population Est. 2019,county,state


In [137]:
df_pop.drop(df_pop.index[3142:3148],inplace=True)
df_pop.shape

(3142, 3)

In [138]:
df_pop = df_pop.rename(columns={'County':'county','State':'state'})
df_pop.head()

,Population Est. 2019,county,state
0,55869.0,Autauga County,Alabama
1,223234.0,Baldwin County,Alabama
2,24686.0,Barbour County,Alabama
3,22394.0,Bibb County,Alabama
4,57826.0,Blount County,Alabama


In [139]:
df_pop_bystate = df_pop.groupby('state').sum()['Population Est. 2019']
df_pop_bystate = df_pop_bystate.to_frame()
df_pop_bystate.reset_index(inplace=True)
df_pop_bystate.head()

,state,Population Est. 2019
0,Alabama,4903185.0
1,Alaska,731545.0
2,Arizona,7278717.0
3,Arkansas,3017804.0
4,California,39512223.0


In [140]:
len(df['county'].unique())

1669

In [141]:
len(df['state'].unique())

55

In [142]:
df.describe()

,cases,deaths
count,86952.000000,86952.000000
mean,167.740098,6.849446
std,2027.119452,127.391531
min,0.000000,0.000000
25%,2.000000,0.000000
50%,8.000000,0.000000
75%,35.000000,1.000000
max,150484.000000,11157.000000


In [143]:
df2 = df.groupby('state').sum()['cases']
df3 = df.groupby('state').sum()['deaths']
df_bystate = pd.concat([df2, df3], axis=1)
df_bystate.reset_index(inplace=True)
df_bystate.head()

,state,cases,deaths
0,Alabama,91383,2650
1,Alaska,7012,143
2,Arizona,94213,3179
3,Arkansas,39459,736
4,California,643866,20182


In [144]:
df_bystate.shape

(55, 3)

In [145]:
df_bystate = pd.merge(df_bystate, df_pop_bystate, on='state')
df_bystate.head()

,state,cases,deaths,Population Est. 2019
0,Alabama,91383,2650,4903185.0
1,Alaska,7012,143,731545.0
2,Arizona,94213,3179,7278717.0
3,Arkansas,39459,736,3017804.0
4,California,643866,20182,39512223.0


In [146]:
df_bystate.shape

(51, 4)

In [147]:
df4 = df.groupby('county').sum()['cases']
df5 = df.groupby('county').sum()['deaths']
df_bycounty = pd.concat([df4, df5], axis=1)
df_bycounty.reset_index(inplace=True)
df_bycounty.head()

,county,cases,deaths
0,Abbeville,293,0
1,Acadia,2371,110
2,Accomack,664,7
3,Ada,12649,190
4,Adair,1774,91


In [148]:
df_bycounty.shape

(1669, 3)

In [149]:
df_bycounty = pd.merge(df_bycounty, df_pop, on='county')
df_bycounty.head()

,county,cases,deaths,Population Est. 2019,state
0,Alexandria city,6001,94,159428.0,Virginia
1,Baltimore city,22964,654,593490.0,Maryland
2,Bethel Census Area,18,0,18386.0,Alaska
3,Bristol city,29,0,16762.0,Virginia
4,Buena Vista city,90,0,6478.0,Virginia


In [150]:
df_bycounty['Pop/1000']=df_bycounty['Population Est. 2019'].div(1000)
df_bycounty.head()

,county,cases,deaths,Population Est. 2019,state,Pop/1000
0,Alexandria city,6001,94,159428.0,Virginia,159.428
1,Baltimore city,22964,654,593490.0,Maryland,593.490
2,Bethel Census Area,18,0,18386.0,Alaska,18.386
3,Bristol city,29,0,16762.0,Virginia,16.762
4,Buena Vista city,90,0,6478.0,Virginia,6.478


In [151]:
df_bycounty['Cases/1000']=df_bycounty['cases'].div(df_bycounty['Pop/1000'])
df_bycounty.head()

,county,cases,deaths,Population Est. 2019,state,Pop/1000,Cases/1000
0,Alexandria city,6001,94,159428.0,Virginia,159.428,37.640816
1,Baltimore city,22964,654,593490.0,Maryland,593.490,38.693154
2,Bethel Census Area,18,0,18386.0,Alaska,18.386,0.979006
3,Bristol city,29,0,16762.0,Virginia,16.762,1.730104
4,Buena Vista city,90,0,6478.0,Virginia,6.478,13.893177


In [152]:
df_bycounty['Deaths/1000']=df_bycounty['deaths'].div(df_bycounty['Pop/1000'])
df_bycounty.head()

,county,cases,deaths,Population Est. 2019,state,Pop/1000,Cases/1000,Deaths/1000
0,Alexandria city,6001,94,159428.0,Virginia,159.428,37.640816,0.589608
1,Baltimore city,22964,654,593490.0,Maryland,593.490,38.693154,1.101956
2,Bethel Census Area,18,0,18386.0,Alaska,18.386,0.979006,0.000000
3,Bristol city,29,0,16762.0,Virginia,16.762,1.730104,0.000000
4,Buena Vista city,90,0,6478.0,Virginia,6.478,13.893177,0.000000


In [153]:
df_bystate['Pop/1000']=df_bystate['Population Est. 2019'].div(1000)
df_bystate.head()

,state,cases,deaths,Population Est. 2019,Pop/1000
0,Alabama,91383,2650,4903185.0,4903.185
1,Alaska,7012,143,731545.0,731.545
2,Arizona,94213,3179,7278717.0,7278.717
3,Arkansas,39459,736,3017804.0,3017.804
4,California,643866,20182,39512223.0,39512.223


In [154]:
df_bystate['Cases/1000']=df_bystate['cases'].div(df_bystate['Pop/1000'])
df_bystate.head()

,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000
0,Alabama,91383,2650,4903185.0,4903.185,18.637477
1,Alaska,7012,143,731545.0,731.545,9.585193
2,Arizona,94213,3179,7278717.0,7278.717,12.943627
3,Arkansas,39459,736,3017804.0,3017.804,13.075402
4,California,643866,20182,39512223.0,39512.223,16.295363


In [155]:
df_bystate['Deaths/1000']=df_bystate['deaths'].div(df_bystate['Pop/1000'])
df_bystate.head()

,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779


In [156]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_geographic_centers_of_the_United_States').text
soup = BeautifulSoup(source, 'html.parser')

In [157]:
My_table = soup.find_all('table',{'class':'wikitable sortable'})[1]

In [158]:
tableheader = My_table.find_all('th')
tableheader

[<th>State or<br/>federal district
 </th>,
 <th>Location
 </th>,
 <th>Coordinates
 </th>]

In [169]:
table_rows = My_table.find_all('tr')
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
df_stateloc = pd.DataFrame(data)
df_stateloc.drop([0],inplace=True)
df_stateloc.drop([1],axis=1,inplace=True)
df_stateloc = df_stateloc.rename(columns={0:'state',2:'coordinates'})
df_stateloc.head()

,state,coordinates
1,Alabama,32°46′46″N 86°49′43″W﻿ / ﻿32.7794°N 86.8287°W﻿...
2,Alaska,64°04′07″N 152°16′42″W﻿ / ﻿64.0685°N 152.2782°...
3,Arizona,34°16′28″N 111°39′37″W﻿ / ﻿34.2744°N 111.6602°...
4,Arkansas,34°53′38″N 92°26′33″W﻿ / ﻿34.8938°N 92.4426°W﻿...
5,California,37°11′03″N 119°28′11″W﻿ / ﻿37.1841°N 119.4696°...


In [170]:
df_bystate = pd.merge(df_bystate, df_stateloc, on='state')
df_bystate.head()

,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,c1,coordinates
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,32°46′46″N 86°49′43″W﻿,32°46′46″N 86°49′43″W﻿ / ﻿32.7794°N 86.8287°W﻿...
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,64°04′07″N 152°16′42″W﻿,64°04′07″N 152°16′42″W﻿ / ﻿64.0685°N 152.2782°...
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,34°16′28″N 111°39′37″W﻿,34°16′28″N 111°39′37″W﻿ / ﻿34.2744°N 111.6602°...
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,34°53′38″N 92°26′33″W﻿,34°53′38″N 92°26′33″W﻿ / ﻿34.8938°N 92.4426°W﻿...
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,37°11′03″N 119°28′11″W﻿,37°11′03″N 119°28′11″W﻿ / ﻿37.1841°N 119.4696°...


In [171]:
df_bystate.shape

(51, 9)

In [173]:
df_bystate['c1'], df_bystate['c2'] = df_bystate['coordinates'].str.split(' / ',1).str
df_bystate.drop(['coordinates'],axis=1,inplace=True)
df_bystate.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,c1,c2
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,32°46′46″N 86°49′43″W﻿,﻿32.7794°N 86.8287°W﻿ / 32.7794; -86.8287﻿ (Ce...
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,64°04′07″N 152°16′42″W﻿,﻿64.0685°N 152.2782°W﻿ / 64.0685; -152.2782﻿ (...
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,34°16′28″N 111°39′37″W﻿,﻿34.2744°N 111.6602°W﻿ / 34.2744; -111.6602﻿ (...
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,34°53′38″N 92°26′33″W﻿,﻿34.8938°N 92.4426°W﻿ / 34.8938; -92.4426﻿ (Ce...
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,37°11′03″N 119°28′11″W﻿,﻿37.1841°N 119.4696°W﻿ / 37.1841; -119.4696﻿ (...


In [174]:
df_bystate.drop(['c1'],axis=1,inplace=True)
df_bystate.head()

,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,c2
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,﻿32.7794°N 86.8287°W﻿ / 32.7794; -86.8287﻿ (Ce...
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,﻿64.0685°N 152.2782°W﻿ / 64.0685; -152.2782﻿ (...
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,﻿34.2744°N 111.6602°W﻿ / 34.2744; -111.6602﻿ (...
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,﻿34.8938°N 92.4426°W﻿ / 34.8938; -92.4426﻿ (Ce...
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,﻿37.1841°N 119.4696°W﻿ / 37.1841; -119.4696﻿ (...


In [175]:
df_bystate['c3'], df_bystate['c4'] = df_bystate['c2'].str.split(' / ',1).str
df_bystate.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,c2,c3,c4
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,﻿32.7794°N 86.8287°W﻿ / 32.7794; -86.8287﻿ (Ce...,﻿32.7794°N 86.8287°W﻿,32.7794; -86.8287﻿ (Center of Alabama)
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,﻿64.0685°N 152.2782°W﻿ / 64.0685; -152.2782﻿ (...,﻿64.0685°N 152.2782°W﻿,64.0685; -152.2782﻿ (Center of Alaska)
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,﻿34.2744°N 111.6602°W﻿ / 34.2744; -111.6602﻿ (...,﻿34.2744°N 111.6602°W﻿,34.2744; -111.6602﻿ (Center of Arizona)
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,﻿34.8938°N 92.4426°W﻿ / 34.8938; -92.4426﻿ (Ce...,﻿34.8938°N 92.4426°W﻿,34.8938; -92.4426﻿ (Center of Arkansas)
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,﻿37.1841°N 119.4696°W﻿ / 37.1841; -119.4696﻿ (...,﻿37.1841°N 119.4696°W﻿,37.1841; -119.4696﻿ (Center of California)


In [176]:
df_bystate.drop(['c2','c3'],axis=1,inplace=True)
df_bystate.head()

,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,c4
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,32.7794; -86.8287﻿ (Center of Alabama)
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,64.0685; -152.2782﻿ (Center of Alaska)
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,34.2744; -111.6602﻿ (Center of Arizona)
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,34.8938; -92.4426﻿ (Center of Arkansas)
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,37.1841; -119.4696﻿ (Center of California)


In [178]:
df_bystate['latitude'], df_bystate['longitude'] = df_bystate['c4'].str.split(' ',1).str
df_bystate.drop(['c4'],axis=1,inplace=True)
df_bystate.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,latitude,longitude
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,32.7794;,-86.8287﻿ (Center of Alabama)
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,64.0685;,-152.2782﻿ (Center of Alaska)
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,34.2744;,-111.6602﻿ (Center of Arizona)
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,34.8938;,-92.4426﻿ (Center of Arkansas)
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,37.1841;,-119.4696﻿ (Center of California)


In [179]:
df_bystate['longitude2'], df_bystate['longitude_text'] = df_bystate['longitude'].str.split(' ',1).str
df_bystate.drop(['longitude_text','longitude'],axis=1,inplace=True)
df_bystate.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,latitude,longitude2
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,32.7794;,-86.8287﻿
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,64.0685;,-152.2782﻿
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,34.2744;,-111.6602﻿
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,34.8938;,-92.4426﻿
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,37.1841;,-119.4696﻿


In [212]:
df_state = df_bystate.copy()
df_state = df_state.rename(columns={'longitude2':'longitude'})
df_state.head()

,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,latitude,longitude
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,32.7794;,-86.8287﻿
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,64.0685;,-152.2782﻿
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,34.2744;,-111.6602﻿
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,34.8938;,-92.4426﻿
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,37.1841;,-119.4696﻿


In [213]:
df_state.dtypes

state                    object
cases                     int64
deaths                    int64
Population Est. 2019    float64
Pop/1000                float64
Cases/1000              float64
Deaths/1000             float64
latitude                 object
longitude                object
dtype: object

In [214]:
df_state['latitude']=df_state['latitude'].astype('str')
df_state['latitude']=df_state['latitude'].str.replace(";","")
df_state['longitude']=df_state['longitude'].astype('str')
df_state.head()

,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,latitude,longitude
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,32.7794,-86.8287﻿
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,64.0685,-152.2782﻿
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,34.2744,-111.6602﻿
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,34.8938,-92.4426﻿
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,37.1841,-119.4696﻿


In [215]:
df_state['latitude'] = pd.to_numeric(df_state['latitude'], errors='coerce')
df_state['longitude'] = df_state['longitude'].str.replace('\ufeff', '').astype('float')
df_state.head()

,state,cases,deaths,Population Est. 2019,Pop/1000,Cases/1000,Deaths/1000,latitude,longitude
0,Alabama,91383,2650,4903185.0,4903.185,18.637477,0.540465,32.7794,-86.8287
1,Alaska,7012,143,731545.0,731.545,9.585193,0.195477,64.0685,-152.2782
2,Arizona,94213,3179,7278717.0,7278.717,12.943627,0.436753,34.2744,-111.6602
3,Arkansas,39459,736,3017804.0,3017.804,13.075402,0.243886,34.8938,-92.4426
4,California,643866,20182,39512223.0,39512.223,16.295363,0.510779,37.1841,-119.4696


In [216]:
lat_us = 39.8333333
lon_us = -98.585522
print('The geographical coordinates of US are {}, {}'.format(lat_us, lon_us))

The geographical coordinates of US are 39.8333333, -98.585522


In [227]:
map_us_cases1000 = folium.Map(location=[lat_us, lon_us], zoom_start=5)

# add markers to map
for lat, lng, case1000, state in zip(df_state['latitude'], df_state['longitude'], df_state['Cases/1000'],df_state['state'] ):
    label = '{}{}{}'.format(case1000,' ',state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=case1000/4,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        parse_html=False).add_to(map_us)  
    
map_us_cases1000

In [233]:
map_us_deaths1000 = folium.Map(location=[lat_us, lon_us], zoom_start=5)

# add markers to map
for lat, lng, death1000, state in zip(df_state['latitude'], df_state['longitude'], df_state['Deaths/1000'],df_state['state']):
    label = '{}{}{}'.format(death1000,' ',state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=death1000*5,
        popup=label,
        color='black',
        fill=True,
        fill_color='grey',
        fill_opacity=0.5,
        parse_html=False).add_to(map_us_deaths1000)  
    
map_us_deaths1000